In [ ]:
import pandas as pd

DATA_PATH = '/kaggle/input/tensorflow-great-barrier-reef/'
train = pd.read_csv(DATA_PATH + 'train.csv')
train

## Basic information about train data

In [ ]:
train.info()

## Check duplicated data

In [ ]:
train.duplicated().sum()

There is no duplicated data in train data

## Feature summary

In [ ]:
def resumetable(df):
    '''function to create feature summary'''
    print(f'Shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['Data Type'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': 'Features'})
    summary['Num of Null Value'] = df.isnull().sum().values
    summary['Num of Unique Value'] = df.nunique().values
    summary['1st Value'] = df.loc[0].values
    summary['2nd Value'] = df.loc[1].values
    summary['3rd Value'] = df.loc[2].values
    return summary

In [ ]:
resumetable(train)

## Downcast

In [ ]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% Compressed'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
train = downcast(train)

## Feature Engineering

In [ ]:
import ast

# Convert String to List Type
train['annotations'] = train['annotations'].apply(ast.literal_eval)

# Get the number of bounding boxes for each image
train['num_bboxes'] = train['annotations'].apply(lambda x: len(x))

In [ ]:
train.head()

## Check number of frames with bounding boxes

In [ ]:
train[train['num_bboxes'] > 0]

In [ ]:
from os import listdir
from PIL import Image

def verify_images(video_id):
    path = DATA_PATH + f'train_images/video_{video_id}/'    
    for filename in listdir(path):
        if filename.endswith('.jpg'):
            try:
                img = Image.open(path + filename)
                img.verify() # Verify it is in fact an image
            except (IOError, SyntaxError) as e:
                print('Bad file:', filename) # Print out the names of corrupt files
    print(f'Video {video_id} has all valid images. Verified!')

for video_id in range(3):
    verify_images(video_id)

## Load sequence of images with annotations

In [ ]:
import numpy as np
from PIL import ImageDraw

def fetch_image(df, video_id, frame_id):
    # get frame
    frame = df[(df['video_id'] == video_id) & (df['video_frame'] == frame_id)].iloc[0]
    # get bounding_boxes
    bounding_boxes = frame['annotations']
    # open image
    img = Image.open(DATA_PATH + f'train_images/video_{video_id}/{frame_id}.jpg')

    for box in bounding_boxes:
        x0, y0, x1, y1 = (box['x'], box['y'], box['x']+box['width'], box['y']+box['height'])
        draw = ImageDraw.Draw(img)
        draw.rectangle( (x0, y0, x1, y1), outline=180, width=5)
    return img

def fetch_image_list(df, video_id, num_images, start_frame_idx):
    image_list = [np.array(fetch_image(df, video_id, start_frame_idx + index)) for index in range(num_images)]

    return image_list

In [ ]:
images = fetch_image_list(train, video_id=0, num_images=80, start_frame_idx=25)

print(f'Number of images: {len(images)}')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 

grid = gridspec.GridSpec(4, 2) 
plt.figure(figsize=(18, 20))

idx_list = [0, 5, 10, 15, 20, 25, 30, 35] 

for i, idx in enumerate(idx_list): 
    ax = plt.subplot(grid[i])
    plt.imshow(images[idx], interpolation='nearest')
    ax.set_title(f'frame index {idx}')
    plt.axis('off')

In [ ]:
from matplotlib import animation, rc
rc('animation', html='jshtml')

def create_animation(imgs, frame_interval=130):
    fig = plt.figure(figsize=(7, 4))
    plt.axis('off')
    img = plt.imshow(imgs[0])

    def animate(i):
        img.set_array(imgs[i])
        return [img]

    return animation.FuncAnimation(fig, animate, frames=len(imgs), interval=frame_interval)

In [ ]:
frame_interval = 130 # set smaller number if you want to play fast, otherwise set bigger

create_animation(images, frame_interval=frame_interval)